<a href="https://colab.research.google.com/github/Ziyu0118/workspace/blob/main/Word2Vec2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
!apt-get install fontconfig -y # Install fontconfig
!fc-cache -fv # Rebuild font cache

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fontconfig is already the newest version (2.13.1-4.2ubuntu5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 2 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/var/cache/fontconfig: cleaning cache directory
/root/.cache/fontconfig: not cleaning non-existent 

In [68]:
import tensorflow as tf
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
matplotlib.use('Agg')
import matplotlib
from pylab import *
mpl.rcParams['font.sans-serif'] = ['DejaVu Sans']
mpl.rcParams['axes.unicode_minus'] = False
import numpy as np
tf.compat.v1.reset_default_graph()
print("step 1")


step 1


In [57]:
sentences = [ "i like dog", "i like cat", "i like animal",
              "dog cat animal", "apple cat dog like", "dog fish milk like",
              "dog cat eyes like", "i like apple", "apple i hate",
              "apple i movie book music like", "cat dog hate", "cat dog like"]
print("step 2")

step 2


In [58]:
word_sequence = " ".join(sentences).split()#得到所有词汇

word_list = " ".join(sentences).split()

word_list = list(set(word_list))#词汇去重
#字典
word_dict = {w: i for i, w in enumerate(word_list)}


print("step 3")

step 3


In [59]:
batch_size = 20#随机选取的数量，为后面训练多轮，每轮选择的个数
embedding_size = 2 # To show 2 dim embedding graph,词向量为2维
voc_size = len(word_list)#13个单词，词汇量个数
print("step 4")

step 4


In [60]:
skip_grams = []
for i in range(1, len(word_sequence) - 1):#除掉第一个字和最后一个字
    target = word_dict[word_sequence[i]]#得到目标字
    context = [word_dict[word_sequence[i - 1]], word_dict[word_sequence[i + 1]]]#上下文字，word_dict为字典{词：数字}

    for w in context:
        skip_grams.append([target, w])#得到[目标字，上一个字],[目标字，下一个字] 即类似[[1, 3], [1, 9],
print("step 5")

step 5


In [61]:
#one-hot标签化
def random_batch(data, size):
    random_inputs = []
    random_labels = []
    np.random.seed(1)
    random_index = np.random.choice(range(len(data)), size, replace=False)#从生成的skip_grams词语对里，随机抽取size个

    for i in random_index:
        random_inputs.append(np.eye(voc_size)[data[i][0]])  # target
        random_labels.append(np.eye(voc_size)[data[i][1]])  # context word

    return random_inputs, random_labels
print("step 6")

step 6


In [62]:
tf.compat.v1.disable_eager_execution() # This line disables eager execution.

# Model
inputs = tf.compat.v1.placeholder(tf.float32, shape=[None, voc_size])#one-hot数组
labels = tf.compat.v1.placeholder(tf.float32, shape=[None, voc_size])#one-hot数组

# W and WT is not Traspose relationship
W = tf.Variable(tf.random.uniform([voc_size, embedding_size], -1.0, 1.0))
WT = tf.Variable(tf.random.uniform([embedding_size, voc_size], -1.0, 1.0))

hidden_layer = tf.matmul(inputs, W) # [batch_size, embedding_size]
output_layer = tf.matmul(hidden_layer, WT) # [batch_size, voc_size]

cost = tf.compat.v1.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output_layer, labels=labels))
optimizer = tf.compat.v1.train.AdamOptimizer(0.001).minimize(cost)

print("step 7")

step 7


In [63]:
with tf.compat.v1.Session() as sess:
    init = tf.compat.v1.global_variables_initializer()
    sess.run(init)

    for epoch in range(5000):
        batch_inputs, batch_labels = random_batch(skip_grams, batch_size)
        _, loss = sess.run([optimizer, cost], feed_dict={inputs: batch_inputs, labels: batch_labels})

        if (epoch + 1)%1000 == 0:
            print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

        trained_embeddings = W.eval()

print("step 8")

Epoch: 1000 cost = 1.409942
Epoch: 2000 cost = 0.986923
Epoch: 3000 cost = 0.902257
Epoch: 4000 cost = 0.865784
Epoch: 5000 cost = 0.844166
step 8


In [69]:
font_names = [f.name for f in fm.fontManager.ttflist]

print(font_names)

for i, label in enumerate(word_list):
    x, y = trained_embeddings[i]
    plt.scatter(x, y)
    plt.annotate(label, xy=(x, y), xytext=(5, 2), textcoords='offset points', ha='right', va='bottom')
plt.title('Word Embedding')
plt.savefig('word_embedding.png')
plt.grid(True)
#plt.show()
plt.close()
print("step 9")

['DejaVu Serif Display', 'DejaVu Serif', 'STIXSizeFiveSym', 'STIXNonUnicode', 'STIXSizeTwoSym', 'DejaVu Serif', 'STIXSizeOneSym', 'cmex10', 'STIXSizeFourSym', 'cmss10', 'STIXNonUnicode', 'DejaVu Sans', 'DejaVu Sans Display', 'DejaVu Sans Mono', 'cmtt10', 'STIXGeneral', 'DejaVu Sans', 'DejaVu Sans', 'DejaVu Serif', 'STIXSizeThreeSym', 'cmb10', 'STIXSizeFourSym', 'DejaVu Sans Mono', 'DejaVu Sans Mono', 'DejaVu Sans Mono', 'STIXNonUnicode', 'cmmi10', 'DejaVu Serif', 'STIXGeneral', 'STIXSizeThreeSym', 'STIXGeneral', 'cmsy10', 'DejaVu Sans', 'STIXGeneral', 'STIXNonUnicode', 'STIXSizeTwoSym', 'STIXSizeOneSym', 'cmr10', 'Liberation Mono', 'Liberation Sans Narrow', 'Liberation Sans', 'Liberation Sans Narrow', 'Liberation Sans', 'Liberation Serif', 'Liberation Mono', 'Liberation Serif', 'Liberation Mono', 'Humor Sans', 'Liberation Serif', 'Liberation Sans', 'Liberation Sans Narrow', 'Liberation Serif', 'Liberation Mono', 'Liberation Sans Narrow', 'Liberation Sans']
step 9
